In [18]:
import dash
import pandas as pd
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [19]:
dataset = pd.read_csv('assets/web_data.csv')
app = dash.Dash(__name__, external_stylesheets=['assets/style.css'])
app.title= "Website analytics"
dataset.head()

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing


In [20]:
colors = ["#f2fffb", "#98ffe0", "#6df0c8", "#59dab2", "#31c194", "#25a27b", "#188463", "#11684d"]
custom_theme = pio.templates["plotly_dark"]
custom_theme.layout.update({"colorway": colors})
pio.templates.default = custom_theme

In [21]:
header = html.Div(
id="header",
    children=[
        html.H4("Website traffic analytics"),
        html.P(
        id="description",
            children="E-commerce website key variables evolution"
        )
    ]
)

In [22]:
visits_per_day=dataset.value_counts("date", sort=False)
visits_per_day.head()

date
2021-10-01    307
2021-10-02    308
2021-10-03    342
2021-10-04    325
2021-10-05    331
Name: count, dtype: int64

In [23]:
trend_title = html.P(className="chart-header", children="Website visits per day")

In [24]:
trend_figure=px.line(visits_per_day, markers=True)
trend_figure.layout.update(showlegend=False)
# trend_figure.show()
trend_graph = dcc.Graph(figure=trend_figure, className="graph")

In [25]:
first_row = html.Div(
className="graph-row",
    children=[
        html.Div(
        className="graph-container",
            children=[trend_title, trend_graph]
        )
    ]
)

In [26]:
funnel_title = html.P(className="chart-header", children="Sales funnel")

In [27]:
@app.callback(Output("funnel-graph", "figure"), [Input("os-dropdown", "value"), Input("country-dropdown", "value")])
def update_funnel_graph(os, country):
    subset = dataset
    if os:
        subset=subset.query(f"os=='{os}'")
    if country:
        subset=subset.query(f"contry=='{country}'")
    visits_per_page = subset.value_counts("page")
    fig=px.funnel(visits_per_page, color=visits_per_page)
    return fig

In [28]:
os_dropdown = dcc.Dropdown(
id="os-dropdown",
    placeholder="Filter by operating system",
    options=[
        {"label": "Windows", "value": "Windows"},
         {"label": "Android", "value": "Android"},
         {"label": "Mac", "value": "Mac"},
         {"label": "iOS", "value": "iOS"},
         {"label": "Linux", "value": "Linux"}
    ]
)

contry_dropdown = dcc.Dropdown(
id="country-dropdown",
    placeholder="Filter by country",
    options=[
        {"label": "United States", "value": "United States"},
        {"label": "Canada", "value": "Canada"}
            ]
)

In [29]:
funnel_controls=html.Div(
    className="controler-row",
    children=["os-dropdown", "country-dropdown"]
)

In [30]:
funnel_graph=dcc.Graph(id="funnel-graph", className="graph")

In [31]:
second_row=html.Div(
    className="graph-row",
    children=[
        html.Div(
            className="graph-container",
            children=[funnel_title, funnel_controls, funnel_graph]
        )
    ]
)

In [32]:
app.layout = html.Div(
    id="root",
children=[header, 
          first_row, second_row
         ]
)

if __name__ == '__main__':
    app.run(mode='inline',  debug=True)